In [7]:
import gograph
import importlib
import re
import time

In [73]:
%%time 
importlib.reload(gograph)

obo_fp = 'data/go-basic.obo'
obo_file = open(obo_fp, 'r')


go_list = {}
blank_template = {'id': None, 'name': None, 'namespace': None,
                  'def': None, 'is_a': []}

for line in obo_file:
    capture = re.search('(.+):\s(.+)\n', line)
    if capture:
        name = capture.group(1)
        content = capture.group(2)
        if name == 'id':
            go_term = gograph.GoTerm(id_=content)
            go_list[content] = go_term # push term into term dict
        elif name == 'is_a':
            go_term.is_a.append(content)
        elif name == 'name':
            go_term.name = content
        elif name == 'namespace':
            go_term.namespace = content
        elif name == 'def':
            go_term.definition = content

obo_file.close()

CPU times: user 1.54 s, sys: 19.9 ms, total: 1.56 s
Wall time: 1.57 s


In [61]:
len(go_list)

47438

In [74]:
go_list['GO:0000001'].__dict__

{'id': 'GO:0000001',
 'namespace': 'biological_process',
 'definition': '"The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton." [GOC:mcc, PMID:10873824, PMID:11389764]',
 'is_a': ['GO:0048308 ! organelle inheritance',
  'GO:0048311 ! mitochondrion distribution'],
 'name': 'mitochondrion inheritance'}

In [31]:
go_list = {}
None in go_list

False

In [34]:
a = 1

if a == 2:
    print('a')
elif a*10 == 10:
    print('b')
else:
    print('c')


b


In [44]:
class test_abc:
    def __init__(self, a=None, b=None):
        self.a = a
        self.b = b

a = test_abc(1, b=2)

print(a.__dict__)

{'a': 1, 'b': 2}


In [56]:
print(1 if 1 < 2 else 'a')

1
